[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/search/question-answering/abstractive-question-answering.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/search/question-answering/abstractive-question-answering.ipynb)

# Abstractive Question Answering

In this notebook we will build an Abstractive question-answering model focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers.

This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A Pinecone vector index to store embeddings and run semantic search
- A retriever model for embedding pieces of text, context passages.
- A generator model to generate answers based on some context provided.

# Install Dependencies

In [1]:
!pip install -qU datasets pinecone-client sentence-transformers torch python-dotenv

### Loading the libraries

In [6]:
from tqdm.auto import tqdm
from pprint import pprint

import torch
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration

import pinecone

from dotenv import load_dotenv
import os

# Load and Prepare Dataset

For this demo we load the the SQUAD dataset in spanish from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

Later we will collect only a few thousands records in order to save time.

In [3]:
# load the dataset from huggingface in and shuffle it
wiki_data = load_dataset(
    'csebuetnlp/xlsum',
    'spanish',
    split='train',
).shuffle(seed=42)

Let's convert our dataset to a pandas Dataframe. To avoid exceding the length of our Pinecone index, we concatenate the summary and the text and extract  only 2,048 characters from each document.

In [4]:
# Convert to pandas
df = wiki_data.to_pandas()
# Join the summary and the text
df['text']=df['summary']+'-'+df['text']
df['text'] = df['text'].str.slice(0,2048)
# select only title and context column
df = df[["title", "summary", "text"]]
# drop rows containing duplicate context passages
df = df.drop_duplicates(subset="summary", ignore_index=True)
df.head(10)

,title,summary,text
0,Crisis en Venezuela: la distorsionada economía...,Hay un lugar en Venezuela rebosante de comida ...,Hay un lugar en Venezuela rebosante de comida ...
1,Alexei Navalny: el líder opositor ruso es dete...,El líder opositor ruso Alexei Navalny fue dete...,El líder opositor ruso Alexei Navalny fue dete...
2,Elecciones en Venezuela: el CNE convoca las co...,La presidenta del Consejo Nacional Electoral (...,La presidenta del Consejo Nacional Electoral (...
3,Las similitudes poco evidentes y escalofriante...,Radicales de extrema derecha en Reino Unido ha...,Radicales de extrema derecha en Reino Unido ha...
4,Argentina: anulan comicios en la provincia de ...,La justicia en Argentina anuló los comicios de...,La justicia en Argentina anuló los comicios de...
5,"Aplican ""gran tirado simultáneo"" de cadena del...",Es una convocatoria a un ejercicio de civismo ...,Es una convocatoria a un ejercicio de civismo ...
6,El ciberacoso que sufren las mujeres que busca...,Buscar un donante de esperma a través de inter...,Buscar un donante de esperma a través de inter...
7,Cinco polícias muertos durante ataque armado e...,Cinco policías resultaron muertos en Egipto du...,Cinco policías resultaron muertos en Egipto du...
8,Cómo ahorrar dinero con 7 simples pasos y no m...,"Aunque parezca difícil, ahorrar es posible.","Aunque parezca difícil, ahorrar es posible.-¿T..."
9,Crisis en Ecuador: 4 razones que explican la c...,Ecuador está pasando por una grave crisis polí...,Ecuador está pasando por una grave crisis polí...


Now, we show some examples:

In [5]:
pprint(df['text'][25])

('El presidente de Venezuela, Nicolás Maduro, quiere mostrar a Estados Unidos '
 'su capacidad de defensa en caso de una eventual intervención militar '
 'extranjera.-El presidente Nicolás Maduro se reunió con miles de '
 'simpatizantes en la llamada "Marcha antiimperialista" de Caracas. Maduro '
 'anunció este lunes que el 26 y 27 de agosto se realizará en todo el país un '
 '"ejercicio de defensa integral armada". Es la primera reacción de Maduro a '
 'las palabras de la semana pasada de su homólogo de Estados Unidos, Donald '
 'Trump, quien dijo que no descartaba una "opción militar" ante el conflicto '
 'social y político que se vive en Venezuela. Maduro habló al final de una '
 'marcha antiimperialista convocada en diversas partes del país. La de '
 'Caracas, multitudinaria, acabó en el palacio de Miraflores con el discurso '
 'del presidente. "Ante la amenaza del emperador Trump, aquí está la respuesta '
 'del pueblo en la calle. ¡Yanquis, go home. Trump, go home! El pueblo de '


In [6]:
pprint(df['text'][100])

('La Asamblea Nacional de Francia votó por legalizar el matrimonio entre '
 'personas del mismo sexo.-La ley fue aprobada por 331 votos a favor contra '
 '225. De esta forma Francia se convierte en décimo cuarto país en aprobar tal '
 'medida. Antes de producirse el voto, hubo escenas de caos en la cámara, con '
 'el orador de orden exigiéndoles a quienes protestaban que salieran del '
 'edificio. La propuesta del presidente francés Francois Hollande de legalizar '
 'el matrimonio homosexual generó manifestaciones a favor y en contra a lo '
 'largo de todo el país en las que participaron cientos de miles de personas. '
 'La nueva ley permite a las parejas homosexuales adoptar niños, algo apoyado '
 'por la mayoría de la población francesa. Final de Quizás también te interese '
 'Partidos de oposición dijeron que apelarán ante el Consejo Constitucional, '
 'el árbitro supremo del país en materia de leyes.')


### Visualize the metadata in the dataset



In [8]:
metadata_cols=['title','text']
# extract batch
batch = df.iloc[0:2]
#get metadata
meta = batch[metadata_cols].to_dict(orient="records")
# Print
print(meta)

[{'title': 'Crisis en Venezuela: la distorsionada economía que crea el oro en el lugar más rico (y violento) del país', 'text': 'Hay un lugar en Venezuela rebosante de comida y de dinero en efectivo.-Aquí, en una semana se gana mucho más que en un mes en otras partes del país. Aquí, el kilo de carne cuesta hasta cuatro veces menos que en Caracas. "Bienvenidos a El Callao. Tierra llena de esperanza y futuro", dice el cartel que saluda a los visitantes de esta localidad en el sur de Venezuela. Se ha convertido en la tierra prometida a la que cada vez acude más gente de todo el país huyendo de la crisis. Parece idílico, ¿verdad? Final de Quizás también te interese No tan rápido. El Callao es un pueblito de casas bajas y suelos ocres rodeado de una verde jungla montañosa. Está ubicado en una de las zonas naturales más ricas de Sudamérica, a unos 850 kilómetros de Caracas. Es conocido por el calipso, un género musical africano-caribeño, y por acoger cada año un carnaval declarado por la Une

# Initialize the Retriever model

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space.
We will use a SentenceTransformer model based on `sentence-transformers/distiluse-base-multilingual-cased-v1` as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [9]:
# set device to GPU if available
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1', device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

From the model final description we know that our embeddings will be 512 tokens length, we need this info to create our vector index in the vector database that we need to store the context passages that we will use to extract the answers.

In [10]:
# Our sentences to encode
sentences = ["This is an example sentence", "Esta es una sentencia de ejemplo"]
# Create the embeddings for our sentences
embeddings = retriever.encode(sentences, convert_to_tensor=True)
# Show the final embeddings
print(embeddings)

tensor([[-0.0389,  0.0185, -0.0407,  ...,  0.0101, -0.0166, -0.0014],
        [-0.0206, -0.0221, -0.0081,  ...,  0.0643,  0.0062,  0.0091]],
       device='cuda:0')


In [11]:
# Calculate distance between this two sentences
distance = util.pytorch_cos_sim(embeddings[0], embeddings[1])
print(distance)

tensor([[0.7632]], device='cuda:0')


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [8]:
# Load .env file with environment variables
load_dotenv()

# connect to pinecone environment
pinecone.init(
    api_key=os.environ["PINECONE_API_KEY"],
    environment="us-west4-gcp-free"  # find next to API key in console
)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 512 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 512-dimension vectors.

In [9]:
index_name = "abstractive-question-answering"

# check if the abstractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=512,
        metric="cosine"
    )

# connect to abstractive-question-answering index we created
index = pinecone.Index(index_name)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [14]:
# we will use batches of 64
batch_size = 64
# In order to minimize compute tim for this demo we limit the number of context passages we will work with
max_context = batch_size*100
print("Max number of context passages:", max_context)

Max number of context passages: 6400


In [15]:
# to avoid exceding the limit in Pinecone metadata for a vecto, remove the text
metadata_cols=['title','text']

# we will use batches of 64
batch_size = 64

# Check if index is empty
index_stats_response = index.describe_index_stats()
if index_stats_response['total_vector_count']<100:
  # for every batch in the dataset
  for i in tqdm(range(0, max_context, batch_size)):
      # find end of batch
      i_end = min(i+batch_size, max_context)
      # extract batch
      batch = df.iloc[i:i_end]
      # generate embeddings for batch
      #emb = retriever.encode(batch["passage_text"].tolist()).tolist()
      emb = retriever.encode(batch["text"].tolist()).tolist()
      # get metadata
      meta = batch[metadata_cols].to_dict(orient="records")
      # create unique IDs
      ids = [f"{idx}" for idx in range(i, i_end)]
      # add all to upsert list
      to_upsert = list(zip(ids, emb, meta))
      # upsert/insert these records to pinecone
      _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 6400}},
 'total_vector_count': 6400}

# Initialize Generator

We will use a `T5-small` variant model, a spanish-T5-small fine-tuned on SQAC for Q&A downstream task. It is a Sequence-To-Sequence model pretrained in spanish. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

Let's initialize the model using transformers.

In [16]:
# load bart tokenizer and model from huggingface
#tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
#generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)
model_name = 'mrm8488/spanish-t5-small-sqac-for-qa'
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [17]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    return xc

In [18]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    #context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    context = [f"<P> {m['metadata']['text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [19]:
#query = "when was the first electric power system built?"
query = "¿Qué presidente de Francia legalizó el matrimonio homosexual?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': '100',
              'metadata': {'text': 'La Asamblea Nacional de Francia votó por '
                                   'legalizar el matrimonio entre personas del '
                                   'mismo sexo.-La ley fue aprobada por 331 '
                                   'votos a favor contra 225. De esta forma '
                                   'Francia se convierte en décimo cuarto país '
                                   'en aprobar tal medida. Antes de producirse '
                                   'el voto, hubo escenas de caos en la '
                                   'cámara, con el orador de orden '
                                   'exigiéndoles a quienes protestaban que '
                                   'salieran del edificio. La propuesta del '
                                   'presidente francés Francois Hollande de '
                                   'legalizar el matrimonio homosexual generó '
                                   'man

In [20]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('question: ¿Qué presidente de Francia legalizó el matrimonio homosexual? '
 'context: <P> La Asamblea Nacional de Francia votó por legalizar el '
 'matrimonio entre personas del mismo sexo.-La ley fue aprobada por 331 votos '
 'a favor contra 225. De esta forma Francia se convierte en décimo cuarto país '
 'en aprobar tal medida. Antes de producirse el voto, hubo escenas de caos en '
 'la cámara, con el orador de orden exigiéndoles a quienes protestaban que '
 'salieran del edificio. La propuesta del presidente francés Francois Hollande '
 'de legalizar el matrimonio homosexual generó manifestaciones a favor y en '
 'contra a lo largo de todo el país en las que participaron cientos de miles '
 'de personas. La nueva ley permite a las parejas homosexuales adoptar niños, '
 'algo apoyado por la mayoría de la población francesa. Final de Quizás '
 'también te interese Partidos de oposición dijeron que apelarán ante el '
 'Consejo Constitucional, el árbitro supremo del país en materia de 

The output looks great. Now let's write a function to generate answers.

In [24]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], padding='max_length', max_length=512, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(input_ids = inputs["input_ids"], attention_mask=inputs['attention_mask'], num_beams=2, min_length=3, max_length=25)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [25]:
generate_answer(query)

'francois hollande'


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [26]:
query = "¿Cuantos policias murieron durante un atraco armado en Egipto?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

'diez policías y militares'


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [27]:
for doc in context["matches"]:
    print(doc["metadata"]["text"], end='\n---\n')

Cinco policías resultaron muertos en Egipto durante un ataque armado en un puesto de control al sur del Cairo, según informó el ministerio del Interior egipcio.-Otros dos resultaron heridos, uno de ellos de gravedad, durante el ataque, que se produjo en la localidad de Beni Suef, a cien kilómetros de la capital. Dos hombres que viajaban en una motocicleta "abrieron ráfagas de fuego" contra los policías que vigilaban el puesto de control, según el comunicado oficial. Las fuerzas de seguridad están peinando el área en busca de los atacantes. Al menos 250 policías y militares han muerto durante ataques armados similares al de este jueves desde que el anterior presidente, Mohamed Morsi, fue depuesto por los militares en julio pasado. Final de Quizás también te interese
---
Siete personas murieron y cuarenta y cuatro resultaron heridas por la explosión de un coche bomba en una estación de policía en la localidad egipcia de El Arish.-Una bomba en El Arish es el último de una serie de ataques

In this case, the answer does not look correct but it is .

If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [28]:
query = "¿Donde se originó el virus COVID-19?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

'en ambientes cerrados'


In [30]:
for doc in context["matches"]:
    print(doc["metadata"]["text"], end='\n---\n')

¿Cuán lejos puede llegar el contagio provocado por el nuevo coronavirus?-Tras varios meses de pandemia de covid-19 ya todos sabemos que uno de los eventos en los que más se propaga el virus son las concentraciones de personas. La Organización Mundial de la Salud (OMS) incluso actualizó sus recomendaciones a fines de mayo en relación a eventos masivos, sugiriendo que, por ejemplo, se realicen al aire libre en vez de en sitios cerrados, que se mantenga el distanciamiento social y que se escalonen los horarios de llegada y salida de los asistentes. Varios países que siguen estas sugerencias todavía mantienen restricciones sobre la realización de reuniones de varias decenas de personas. Pero si crees que el peligro de los contagios solo está en los eventos masivos, puede que la siguiente visualización te haga pensar diferente. Final de Quizás también te interese Un caso testigo El esquema que acabas de ver pertenece a un caso testigo que informó el Departamento de Salud del condado de Cata

Let’s finish with a final few questions.

In [31]:
query = "¿Quién es el presidente de Venezuela?"
context = query_pinecone(query, top_k=1)
query = format_query(query, context["matches"])
generate_answer(query)

'nicolás maduro'


As we can see, the model can answer correctly but iit is not very "abstractive". Answers are short, rigth to the point.